<a href="https://colab.research.google.com/github/Peantoo/DE-Notebooks/blob/main/DE_Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy

import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
np.random.seed(0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Data

**TODO**

1.   Identify the encoding of the data in filename
2.   Read the csv into suicide_attacks variable using the correct encoding (the chardet module might come handy).


In [ ]:
filename = "/content/drive/MyDrive/Colab Notebooks/Data Sets/PakistanSuicideAttacks Ver 11 (30-November-2017).csv"

In [ ]:
rawdata = open(filename, "rb").read() #Read as binary 
enc_type = chardet.detect(rawdata) #Figure out encoding type

In [ ]:
encoding = enc_type.get('encoding')

suicide_attacks = pd.read_csv((filename), encoding=encoding)
suicide_attacks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   S#                       496 non-null    int64  
 1   Date                     496 non-null    object 
 2   Islamic Date             342 non-null    object 
 3   Blast Day Type           486 non-null    object 
 4   Holiday Type             72 non-null     object 
 5   Time                     285 non-null    object 
 6   City                     496 non-null    object 
 7   Latitude                 493 non-null    float64
 8   Longitude                493 non-null    object 
 9   Province                 496 non-null    object 
 10  Location                 493 non-null    object 
 11  Location Category        461 non-null    object 
 12  Location Sensitivity     460 non-null    object 
 13  Open/Closed Space        461 non-null    object 
 14  Influencing Event/Event  1

#Preliminary text pre-processing

**TODO**




1.   Clean the City column for inconsisntecies
2.   Normalize the City column for upper or lowercase, spaces, etc.



In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',

Obviously a lot of errors. Many incorrect duplicates due to poor capitalization, poor spelling, poor punctuation, trailing spaces, it's a nightmare. Best to start from a clean slate. Make everything lowercase, remove trailing spaces, remove dupes, then apply name formatting. Then we can look for outliers. 

#Matching of inconsistent data entries

**TODO**



1.   Verify there are no more inconsistencies in the City column.
2.   Feel free to use the fuzzywuzzy package to match an remove possible issues.






>  Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.



In [ ]:
cities = suicide_attacks['City']
cities = cities.str.lower()
cities = cities.str.strip()
cities = cities.unique()

cities.sort()

In [ ]:
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad',
       'jacobabad', 'karachi', 'karak', 'khanewal', 'khuzdar',
       'khyber agency', 'kohat', 'kuram agency', 'kurram agency',
       'lahore', 'lakki marwat', 'lasbela', 'lower dir', 'malakand',
       'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=

Looks better but there's some obvious issues, like D.I Khan, D, I Khan, D.G Khan, etc. For some of them I'll need to look up the cities because I don't know which of these are real or not.

In [ ]:
# sort them alphabetically and then take a closer look
cities2=cities.astype(str)

print("\n".join(cities))

attock
bajaur agency
bannu
bhakkar
buner
chakwal
chaman
charsadda
d. i khan
d.g khan
d.i khan
dara adam khel
fateh jang
ghallanai, mohmand agency
gujrat
hangu
haripur
hayatabad
islamabad
jacobabad
karachi
karak
khanewal
khuzdar
khyber agency
kohat
kuram agency
kurram agency
lahore
lakki marwat
lasbela
lower dir
malakand
mansehra
mardan
mohmand agency
mosal kor, mohmand agency
multan
muzaffarabad
north waziristan
nowshehra
orakzai agency
peshawar
pishin
poonch
quetta
rawalpindi
sargodha
sehwan town
shabqadar-charsadda
shangla
shikarpur
sialkot
south waziristan
sudhanoti
sukkur
swabi
swat
taftan
tangi, charsadda district
tank
taunsa
tirah valley
totalai
upper dir
wagah
zhob


This is obvious a case for fuzzy matching. Any time you need to correct values where there's typos or small variances, using fuzzy matching to replace is a simple solution. Unfortunately, my experience with NLP is less hands-on. I tried for a long time to come up with a unique solution, but sometimes, just Googling a coding problem and using Stack Exchange or existing answers is the way to go. 

Could I have written the code below on my own? Sure, but in this case, that's inefficient and I don't have enough experience with fuzzy matching to do it quickly nor the time to spend on it.
 


In [ ]:
matches = fuzzywuzzy.process.extract("d. i khan", cities, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

In [ ]:
print(matches)

[('d. i khan', 100), ('d.i khan', 100), ('d.g khan', 88), ('khanewal', 50), ('sudhanoti', 47)]


Obviously D.I Khan is the same, but I don't know what D.I or D.G stands for, so I had to look it up. Apparently they should both be "D.I. Khan," which is Dera Ismail Khan. D.G. Khan is Dera Ghazi Khan, so it is not a typo. I should keep my ratio to replace above 90%.

In [ ]:
moh_matches = fuzzywuzzy.process.extract("mohmand agency", cities, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(moh_matches)

[('mohmand agency', 100), ('ghallanai, mohmand agency', 74), ('mosal kor, mohmand agency', 74), ('orakzai agency', 64), ('kuram agency', 62)]


Interestingly, there are occasionally entries for an entire area as well as cities within that area, like "charsadda / tangi, charsadda" and "mohmand agency / mosal kor, mohmand agency." Fuzzy matching wouldn't be good for combining these, since it's not sure whether we actually want them to be the same. Do I need to combine them? Do I need to keep them separate? The answers to these questions determine what I can set my min_ratio at. Sometimes using judgement and reasoning is still necessary.

In [ ]:
# Function to replace rows in the provided column of the provided dataframe
# That match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
       strings = df[column].unique()
    
       matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

       close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

       rows_with_matches = df[column].isin(close_matches)

       df.loc[rows_with_matches, column] = string_to_match
    
       print("All done!")

In [ ]:
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="d.i khan")

All done!


In [ ]:
# my previous code didn't directly alter the original file, so I will need to reapply those transformations here.
suicide_attacks['City'] = suicide_attacks['City'].str.lower()
suicide_attacks['City'] = suicide_attacks['City'].str.strip()
cities = suicide_attacks['City'].unique()
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)